In [4]:
import pandas as pd
import numpy as np
from HullWhite import HullWhite
from scipy.optimize import fsolve
S = np.arange(0,11)
T = np.arange(0,10.5,0.5)
beta = np.array([1.879892, 0.579211, 	3.993992, 1.445091])
tau  = np.array([ 16.633491, 	0.319680])
reversion=0.13949636660880768 
volatility=0.017793899652989272
# HW = HullWhite(initial=0.02459103, reversion=0.03, volatility=0.00200, Gamma=1000, b=beta, tau=tau)
HW = HullWhite(initial=0.02459103, reversion=reversion, volatility=volatility, b=beta, tau=tau)
Kswap=fsolve(lambda x: HW.swap(0, S, T, x), x0=0.02)[0]
Kswpt=fsolve(lambda x: HW.swap(0, S+5, T+5, x), x0=0.02)[0]
time,float = HW.create_path(1/365,10,0)
swap = HW.swapextended(0, S, T, Kswap, floatRate=float, schedule=time)
swpt = HW.swaption(0, Te=5, S=S+5, T=T+5, K=Kswpt, floatRate=float, schedule=time)
simTable = pd.read_csv('SimulationTimesSpread.txt')
simTable['Time'] = round(simTable['Time'],2)
simTable['CVACONF'] = round((simTable['CVAUB']/simTable['CVA']-1)*100,4)
simTable['DVACONF'] = round((simTable['DVALB']/simTable['DVA']-1)*100,4)
simTable['CVA'] = simTable['CVA']*1000000
simTable['DVA'] = simTable['DVA']*1000000
simTable['COLVA'] = simTable['COLVA']*1000000
simTable['CVAUB'] = simTable['CVAUB']*1000000
simTable['CVALB'] = simTable['CVALB']*1000000
simTable['DVAUB'] = simTable['DVAUB']*1000000
simTable['DVALB'] = simTable['DVALB']*1000000

# simTable.drop(['CVAUB', 'CVALB', 'DVAUB', 'DVALB'], axis=1, inplace=True)
# simTable['V0']=np.where('Swaption' in simTable['Product'], "swpt", "swap")
simTable['V0'] = np.where(simTable['Product'].str.contains('Swaption'), 'swpt', np.where(simTable['Product'].str.contains('Forward'), 'fwd', 'swap'))
simTable['VM'] = np.where(simTable['Product'].str.contains('VM'), 1, 0)
simTable['IM'] = np.where(simTable['Product'].str.contains('IM'), 1, 0)
simTable.sort_values(['V0', 'VM', 'IM', 'dt'], ascending=True, inplace=True)
simTable.reset_index(drop=True, inplace=True)
# simTable.drop_duplicates(subset=['N','Product', 'dt'], keep='last', inplace=True)
# simTable['normTime'] = simTable['Time']/simTable['dt']
simTable

,N,dt,CVA,CVAUB,CVALB,DVA,DVAUB,DVALB,COLVA,SPREAD,Product,Time,CVACONF,DVACONF,V0,VM,IM
0,10000,365,-159.600421,-170.725975,-148.474868,94.300615,88.161910,100.439321,110.872815,0.0025,WithSpread10Y Payer Swap with VM,3155.05,6.9709,6.5097,swap,1,0
1,10000,365,-2.752282,-3.043004,-2.461560,0.885831,0.839544,0.932118,-306.531783,0.0025,WithSpread10Y Payer Swap with VM+IM,3096.97,10.5629,5.2253,swap,1,1
